<a href="https://colab.research.google.com/github/HomayounfarM/Interactive-data-visualization/blob/main/plotly/Ex1_plotly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reading data

The treemap in a CSV file

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from google.colab import files
from io import StringIO
import io

In [ ]:
# Upload the file
uploaded = files.upload()

# Extract the filename
for filename in uploaded.keys():
    # Read the CSV file
    df_tree = pd.read_csv(io.StringIO(uploaded[filename].decode('utf-8')))

df_tree

Saving hierarchical_data.csv to hierarchical_data (4).csv


,names,parents
0,Mainbox,NaN
1,box 1,Mainbox
2,box 2,Mainbox
3,box 3,Mainbox
4,box 4,Mainbox
5,Sub-box 1.1,box 1
6,Sub-box 1.2,box 1
7,Sub-box 2.1,box 2
8,Sub-box 2.2,box 2
9,Sub-box 2.3,box 2


# 1- Default Clicking: Handled internally by Plotly

Clicking on a Node: By default, when you click on a node in a Treemap, Plotly's JavaScript library manages the interaction. It will zoom into the clicked node, showing its children and collapsing other parts of the hierarchy.

In [ ]:
import pandas as pd
import plotly.express as px

# Create a Treemap using Plotly
fig = px.treemap(
    df_tree,
    names = df_tree.names,                   # Specify the names of the nodes
    parents = df_tree.parents                # Specify the parent-child relationships
)

# Customize the Treemap to show labels up to the second parental level
fig.update_traces(
    root_color="lightgrey",
    textinfo="label",  # Show labels and percentage entry
    texttemplate="%{label}",  # Customize text template to control what is displayed
    textfont=dict(size=14)  # Adjust font size if needed
)

# Update layout to customize the font
fig.update_layout(
  font_family='Open Sans',
  font_size=32,
  title={'text': "Hierarchical Tree Structure", 'x': 0.5, 'xanchor': 'center'},
  margin=dict(t=60, l=25, r=25, b=25))  # Adjust margins for better fit

# Show the interactive treemap
fig.show()

In [ ]:
# Add a level column to determine which levels to label
def add_levels(df):
    # Initialize the level column
    df['level'] = -1

    # Calculate levels for each node
    for index, row in df.iterrows():
        level = 0
        parent = row['parents']
        while parent in df['names'].values:
            level += 1
            parent = df[df['names'] == parent]['parents'].values[0]
        df.at[index, 'level'] = level

    return df

df_tree = add_levels(df_tree)

# Create a Treemap using Plotly
fig = px.treemap(
    df_tree,
    names='names',        # Specify the column name for node names
    parents='parents',    # Specify the column name for parent-child relationships
    #color='level',        # Use the level column for color (optional)
    color_continuous_scale='Viridis'  # Optional: customize color scale
)

# Customize labels and other properties
fig.update_traces(
    root_color="lightgrey",
    textinfo="label",  # Show labels only
    texttemplate="%{label}",  # Display only the label
    textfont=dict(size=14)   # Adjust font size if needed
)

# Update layout to customize the font and other properties
fig.update_layout(
    font_family='Open Sans',
    font_size=32,
    title={'text': "Hierarchical Tree Structure", 'x': 0.5, 'xanchor': 'center'},
    margin=dict(t=60, l=25, r=25, b=25)  # Adjust margins for better fit
)

# Show the interactive Treemap
fig.show()

In [ ]:
pip install dash

# 2- Custom Handling: Implemented via JavaScript for web apps or with additional tools like Dash for Python applications.

In a Jupyter Notebook or Python environment, you can capture click events using the plotly library's callback mechanism, but this requires additional setup, often involving dash or ipywidgets for more interactive behaviors.

Dash is a popular framework for building interactive web applications in Python, and it is used for creating dynamic and responsive interfaces.

In [ ]:
# Import necessary libraries
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

df = df_tree.copy()

# Add a level column to determine which levels to label
# Function to add levels
def add_levels(df):
    df['level'] = -1
    for index, row in df.iterrows():
        level = 0
        parent = row['parents']
        while parent in df['names'].values:
            level += 1
            parent = df[df['names'] == parent]['parents'].values[0]
        df.at[index, 'level'] = level
    return df

df_tree = add_levels(df_tree)

# Create a Treemap using Plotly
fig = px.treemap(
    df_tree,
    names='names',
    parents='parents',
    #color='level',  # Optional: Use color to differentiate levels
    color_continuous_scale='Viridis'
)

# Customize labels and other properties
fig.update_traces(
    root_color="lightgrey",
    textinfo="label",  # Show labels and percentage entry
    texttemplate="%{label}",  # Customize text template to control what is displayed
    textfont=dict(size=14)  # Adjust font size if needed
)

# Update the texttemplate based on the level
def update_texttemplate(trace, levels_to_show=[0, 1]):
    for i, level in enumerate(trace.customdata[:2, 0]):
        if level in levels_to_show:
            trace.texttemplate[i]
            # trace.texttemplate[i] = "%{label}"
        else:
            trace.texttemplate[i] = ""  # Hide labels for other levels
# Adding customdata to use levels for updating texttemplate
fig.update_traces(
    customdata=df_tree[['level']].to_numpy(),  # Add level information
    textinfo="label+text",
    texttemplate="%{label}"
)

# Apply custom texttemplate based on level
for trace in fig.data:
    update_texttemplate(trace)

# Update layout to customize the font and other properties
fig.update_layout(
    font_family='Open Sans',
    font_size=32,
    title={'text': "Hierarchical Tree Structure", 'x': 0.5, 'xanchor': 'center'},
    margin=dict(t=60, l=25, r=25, b=25)
)

# Initialize Dash app
app = Dash(__name__)

# Define app layout
app.layout = html.Div([
    dcc.Graph(
        id='treemap',
        figure=fig
    ),
    html.Div(id='click-data')
])

# Define callback to update the click-data Div
@app.callback(
    Output('click-data', 'children'),
    Input('treemap', 'clickData')
)

def update_click_data(clickData):
    if clickData:
        node_name = clickData['points'][0]['label']
        return f'Clicked on node: {node_name}'
    else:
        return 'Click a node in the Treemap'

# Run the server
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>

In [ ]:
# Import necessary libraries
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

df = df_tree.copy()

# Add a level column to determine which levels to label
def add_levels(df):
    df['level'] = -1
    for index, row in df.iterrows():
        level = 0
        parent = row['parents']
        while parent in df['names'].values:
            level += 1
            parent = df[df['names'] == parent]['parents'].values[0]
        df.at[index, 'level'] = level
    return df

df_tree = add_levels(df_tree)

# Create a Treemap using Plotly
fig = px.treemap(
    df_tree,
    names='names',
    parents='parents',
    color='level',  # Use color to differentiate levels
    color_continuous_scale='Viridis'
)

# Initialize Dash app
app = Dash(__name__)

# Define app layout
app.layout = html.Div([
    dcc.Graph(
        id='treemap',
        figure=fig
    ),
    html.Div(id='click-data')
])

# Define callback to update the treemap layout based on node clicks
@app.callback(
    Output('click-data', 'children'),
    Input('treemap', 'clickData')
)
def update_click_data(clickData):
    if clickData:
        try:
            node_name = clickData['points'][0]['label']
            return f'Clicked on node: {node_name}'
        except (KeyError, IndexError) as e:
            return 'Error processing click data'
    else:
        return 'Click a node in the Treemap'

# Run the server
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>

# Test0

In [1]:
# Import necessary libraries
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

df = df_tree.copy()

# Add a level column to determine which levels to label
def add_levels(df):
    df['level'] = -1
    for index, row in df.iterrows():
        level = 0
        parent = row['parents']
        while parent in df['names'].values:
            level += 1
            parent = df[df['names'] == parent]['parents'].values[0]
        df.at[index, 'level'] = level
    return df

df_tree = add_levels(df_tree)

# Create initial Treemap using Plotly
show_level_4_labels = True
print(len(fig.data[0]))
fig.data[0].textinfo ='text+label'
def create_treemap(show_level_4_labels=False):
    fig = px.treemap(
        df_tree,
        names='names',
        parents='parents',
        color='level',  # Use color to differentiate levels
        color_continuous_scale='Viridis')
        # Adjust label visibility
    if show_level_4_labels:
        df_tree.loc[df_tree['level']==4,'text+lable']='text+label'
        fig.data[0].textinfo = 'text'
        #fig.data[0].textinfo = df_tree['text+label']
    return fig

# Initialize Dash app
app = Dash(__name__)

# Define app layout
app.layout = html.Div([
    dcc.Graph(
        id='treemap',
        figure=create_treemap(show_level_4_labels=True)
    ),
    html.Div(id='click-data')
])

# Define callback to update the treemap layout based on node clicks

def update_treemap(clickData):
    if clickData:
        node_name = clickData['points'][0]['label']
        if df_tree[df_tree['names'] == node_name]['level'].values[0] == 0:
            fig = create_treemap(show_level_4_labels=True)
        else:
            fig = create_treemap(show_level_4_labels=True)
    return fig

@app.callback(
    Output('click-data', 'children'),
    Input('treemap', 'clickData')
)
def update_click_data(clickData):
    if clickData:
        try:
            node_name = clickData['points'][0]['label']
            mydummy = df_tree[df_tree['names'] == node_name]['level'].values[0]
            return f'Clicked on node: {node_name}', f"Level number:{mydummy}"
        except (KeyError, IndexError) as e:
            return 'Error processing click data'
    else:
        return 'Click a node in the Treemap'

# Run the server
if __name__ == '__main__':
    app.run_server(debug=True)

ModuleNotFoundError: No module named 'dash'

In [ ]:
df_tree['text+label']

,text+label
0,text
1,text
2,text
3,text
4,text
5,text
6,text
7,text
8,text
9,text


In [ ]:
fig.data[0]

Treemap({
    'domain': {'x': [0.0, 1.0], 'y': [0.0, 1.0]},
    'hovertemplate': 'label=%{label}<br>parent=%{parent}<br>level=%{color}<extra></extra>',
    'labels': array(['Mainbox', 'box 1', 'box 2', 'box 3', 'box 4', 'Sub-box 1.1',
                     'Sub-box 1.2', 'Sub-box 2.1', 'Sub-box 2.2', 'Sub-box 2.3',
                     'Sub-box 3.1', 'Sub-box 3.2', 'Sub-box 4.1', 'Sub-box 4.2',
                     'Sub-box 4.3', 'Sub-sub-box 1.1.1', 'Sub-sub-box 1.1.2',
                     'Sub-sub-box 1.2.1', 'Sub-sub-box 1.2.2', 'Sub-sub-box 2.1.1',
                     'Sub-sub-box 2.1.2', 'Sub-sub-box 2.2.1', 'Sub-sub-box 2.2.2', nan, nan,
                     nan, nan, nan, nan, nan, nan, nan, nan, nan, 'Sub-sub-sub-box 1.1.1.1',
                     'Sub-sub-sub-box 1.1.1.2', 'Sub-sub-sub-box 1.1.1.3'], dtype=object),
    'marker': {'coloraxis': 'coloraxis',
               'colors': array([0, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3,
                  

# Test

In [ ]:
# Import necessary libraries
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

df = df_tree.copy()

# Add a level column to determine which levels to label
def add_levels(df):
    df['level'] = -1
    for index, row in df.iterrows():
        level = 0
        parent = row['parents']
        while parent in df['names'].values:
            level += 1
            parent = df[df['names'] == parent]['parents'].values[0]
        df.at[index, 'level'] = level
    return df

df_tree = add_levels(df_tree)

# Create initial Treemap using Plotly
def create_treemap(show_level_4_labels=False):
    fig = px.treemap(
        df_tree,
        names='names',
        parents='parents',
        color='level',  # Use color to differentiate levels
        color_continuous_scale='Viridis'
    )
    # Adjust label visibility
    for i, trace in enumerate(fig.data):
        if trace.name == 'root':
            trace.textinfo = 'label+text'
        elif show_level_4_labels and trace.name == 'level_4':
            trace.textinfo = 'label+text'
        else:
            trace.textinfo = 'label'
    return fig

# Initialize Dash app
app = Dash(__name__)

# Define app layout
app.layout = html.Div([
    dcc.Graph(
        id='treemap',
        figure=create_treemap(show_level_4_labels=False)
    ),
    html.Div(id='click-data')
])

# Define callback to update the treemap layout based on node clicks
@app.callback(
    Output('click-data', 'children'),
    Input('treemap', 'clickData')
)
def update_treemap(clickData):
    if clickData:
        node_name = clickData['points'][0]['label']
        # Check if clicked node is a level 1 node
        if df_tree[df_tree['names'] == node_name]['level'].values[0] == 1:
            # Show labels for level 4
            fig = create_treemap(show_level_4_labels=True)
        else:
            # Hide labels for level 4
            fig = create_treemap(show_level_4_labels=False)
    else:
        fig = create_treemap(show_level_4_labels=False)
    return fig

def update_click_data(clickData):
    if clickData:
        try:
            node_name = clickData['points'][0]['label']
            return f'Clicked on node: {node_name}'
        except (KeyError, IndexError) as e:
            return 'Error processing click data'
    else:
        return 'Click a node in the Treemap'

# Run the server
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>

In [ ]:
import plotly.express as px
import pandas as pd

# Example DataFrame
df_tree = pd.DataFrame({
    'names': ['root', 'child1', 'child2', 'subchild1', 'subchild2'],
    'parents': ['', 'root', 'root', 'child1', 'child2'],
    'level': [0, 1, 1, 2, 2]
})

# Create the treemap figure
fig = px.treemap(df_tree, names='names', parents='parents', color='level')

# Access the single trace
trace = fig.data[0]

# Adjust textinfo based on some condition
show_level_4_labels = False

if show_level_4_labels:
    trace.textinfo = 'label'  # Show both labels and text for all levels
else:
    # Show only labels for levels other than level 4
    # Note: Adjusting visibility based on levels requires custom logic
    trace.textinfo = 'text'  # Hide additional text information

# Update the figure with the adjusted trace
fig.update_traces(textinfo=trace.textinfo)

# Display the figure
fig.show()


In [ ]:
fig.data[0]

Treemap({
    'domain': {'x': [0.0, 1.0], 'y': [0.0, 1.0]},
    'hovertemplate': 'label=%{label}<br>parent=%{parent}<br>level=%{color}<extra></extra>',
    'labels': array(['root', 'child1', 'child2', 'subchild1', 'subchild2'], dtype=object),
    'marker': {'coloraxis': 'coloraxis', 'colors': array([0, 1, 1, 2, 2])},
    'name': '',
    'parents': array(['', 'root', 'root', 'child1', 'child2'], dtype=object),
    'textinfo': 'text'
})

array(['root', 'child1', 'child2', 'subchild1', 'subchild2'], dtype=object)